In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itbr" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:01<00:00,  5.10s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:   0%|          | 0/26 [00:00<?, ?batch/s, loss=0.29] 

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9140 | AUPR: 0.6372 | Acc: 95.23%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9179 | AUPR: 0.6523 | Acc: 95.40%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9193 | AUPR: 0.6599 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9132 | AUPR: 0.6415 | Acc: 95.24%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9187 | AUPR: 0.6595 | Acc: 95.39%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6687 | Acc: 95.36%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6687 | Acc: 95.36%
Internal test: AUC: 0.9221 | AUPR: 0.6847 | Acc: 95.09%
External test: AUC: 0.9188 | AUPR: 0.6637 | Acc: 95.08%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9153 | AUPR: 0.6202 | Acc: 94.89%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9150 | AUPR: 0.6248 | Acc: 94.98%
model(kernel=linear, C=1)
Valid:         AUC: 0.9150 | AUPR: 0.6281 | Acc: 95.05%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9153 | AUPR: 0.6202 | Acc: 94.89%
Internal test: AUC: 0.9162 | AUPR: 0.6381 | Acc: 94.65%
External test: AUC: 0.9115 | AUPR: 0.5937 | Acc: 94.39%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7573 | AUPR: 0.4131 | Acc: 94.45%
model(n_neighbors=5)
Valid:         AUC: 0.7852 | AUPR: 0.4592 | Acc: 94.74%
model(n_neighbors=7)
Valid:         AUC: 0.8080 | AUPR: 0.4922 | Acc: 94.84%
best model(n_neighbors=7)
Valid:         AUC: 0.8080 | AUPR: 0.4922 | Acc: 94.84%
Internal test: AUC: 0.8026 | AUPR: 0.5120 | Acc: 94.82%
External test: AUC: 0.8066 | AUPR: 0.4933 | Acc: 94.71%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8715 | AUPR: 0.4286 | Acc: 83.50%
best model()
Valid:         AUC: 0.8715 | AUPR: 0.4286 | Acc: 83.50%
Internal test: AUC: 0.8785 | AUPR: 0.4486 | Acc: 83.99%
External test: AUC: 0.8677 | AUPR: 0.4051 | Acc: 82.23%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9169 | AUPR: 0.6517 | Acc: 95.27%
model(C=10.0)
Valid:         AUC: 0.9168 | AUPR: 0.6523 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.9164 | AUPR: 0.6518 | Acc: 95.29%
best model(C=1.0)
Valid:         AUC: 0.9169 | AUPR: 0.6517 | Acc: 95.27%
Internal test: AUC: 0.9201 | AUPR: 0.6737 | Acc: 95.06%
External test: AUC: 0.9117 | AUPR: 0.6300 | Acc: 94.92%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8797 | AUPR: 0.5502 | Acc: 83.53%
model(criterion=log_loss)
Valid:         AUC: 0.8877 | AUPR: 0.5736 | Acc: 84.91%
model(criterion=entropy)
Valid:         AUC: 0.8877 | AUPR: 0.5736 | Acc: 84.91%
best model(criterion=log_loss)
Valid:         AUC: 0.8877 | AUPR: 0.5736 | Acc: 84.91%
Internal test: AUC: 0.8765 | AUPR: 0.6061 | Acc: 84.72%
External test: AUC: 0.8829 | AUPR: 0.5957 | Acc: 84.51%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9166 | AUPR: 0.6503 | Acc: 89.01%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9177 | AUPR: 0.6528 | Acc: 89.12%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9182 | AUPR: 0.6541 | Acc: 89.16%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9161 | AUPR: 0.6522 | Acc: 88.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9179 | AUPR: 0.6548 | Acc: 89.06%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9185 | AUPR: 0.6560 | Acc: 89.12%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9161 | AUPR: 0.6522 | Acc: 88.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9179 | AUPR: 0.6548 | Acc: 89.06%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9185 | AUPR: 0.6560 | Acc: 89.12%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9185 | AUPR: 0.6560 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9196 | AUPR: 0.6501 | Acc: 95.20%
model(criterion=squared_error)
Valid:         AUC: 0.9196 | AUPR: 0.6498 | Acc: 95.18%
best model(criterion=squared_error)
Valid:         AUC: 0.9196 | AUPR: 0.6498 | Acc: 95.18%
Internal test: AUC: 0.9191 | AUPR: 0.6598 | Acc: 95.06%
External test: AUC: 0.9195 | AUPR: 0.6510 | Acc: 95.04%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9156 | AUPR: 0.6315 | Acc: 95.21%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9176 | AUPR: 0.6364 | Acc: 95.29%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9178 | AUPR: 0.6401 | Acc: 95.25%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9178 | AUPR: 0.6401 | Acc: 95.25%
Internal test: AUC: 0.9196 | AUPR: 0.6764 | Acc: 95.11%
External test: AUC: 0.9118 | AUPR: 0.6213 | Acc: 94.81%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)